# GPT-2 Using HuggingFance and Tensorflow
En este Notebook se realizarán los procesamientos necesarios para hacer fine tuning de GPT-2 para la generación de memes en español.


## 1. Carga de Información
Durante la primera fase de creación, se utilizará únicamente uno de los conjuntos de datos disponibles. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/bases_no_tilde/useful_scrapymemes1m_no_tilde.csv')

In [3]:
df = df[(~df.text_1.isna()) & (~df.text_2.isna())]
df["text_1"] = df["text_1"].str.lower()
df["text_2"] = df["text_2"].str.lower()
df['meme_id'] = df['meme_id'].apply('{:0>4}'.format)

In [4]:
df.head()

,Unnamed: 0,meme_id,plantilla,texto_no_tilde,text_1,text_2
0,4,0007,Futurama Fry,no se si cortarme las venas o dejarmelas largas,no se si cortarme las venas,o dejarmelas largas
1,13,0007,Futurama Fry,no se si estudiar control o estudiar elementos,no se si estudiar control,o estudiar elementos
3,46,0003,Ay Si,ay si me pongo copete y ya soy emo,ay si,me pongo copete y ya soy emo
4,50,0012,Willy Wonka,¿conque eres regaytonero?... !!dime que se sie...,¿conque eres regaytonero?... !!,dime que se siente cagar por la boca..
5,51,0007,Futurama Fry,nO estoy seguro si nadie se levanta temprano o...,no estoy seguro si nadie se levanta temprano,o no saben que entra en el control


In [12]:
def GPT_text(row):
    meme_id = row.meme_id
    text_1 = row.text_1 
    text_2 = row.text_2 
    return '<|'+meme_id+'|>' + text_1.strip() + ' | ' + text_2.strip()

df['GPT_text'] = df.apply(GPT_text, axis = 1)

In [13]:
df[['GPT_text', 'meme_id', 'text_1', 'text_2']].head()

,GPT_text,meme_id,text_1,text_2
0,<|0007|>no se si cortarme las venas | o dejarm...,0007,no se si cortarme las venas,o dejarmelas largas
1,<|0007|>no se si estudiar control | o estudiar...,0007,no se si estudiar control,o estudiar elementos
3,<|0003|>ay si | me pongo copete y ya soy emo,0003,ay si,me pongo copete y ya soy emo
4,<|0012|>¿conque eres regaytonero?... !! | dime...,0012,¿conque eres regaytonero?... !!,dime que se siente cagar por la boca..
5,<|0007|>no estoy seguro si nadie se levanta te...,0007,no estoy seguro si nadie se levanta temprano,o no saben que entra en el control


In [18]:
train_text, test_text, train_id, test_id  = train_test_split(df['GPT_text'], df['meme_id'], train_size=.9, random_state=42) # Para hacerlo estratificado

In [19]:
print(f'Memes en Train: {len(train_text)}\nMemes en Test: {len(test_text)}')

Memes en Train: 129019
Memes en Test: 14336


In [ ]:
model_data_path = '../model_data'

train = os.path.join(model_data_path, 'train_trial.txt')
test = os.path.join(model_data_path, 'test_trial.txt')

with open(train, 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train_text.values.tolist()))

with open(test, 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(test_text.values.tolist()))

In [9]:
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer


tokenizer = GPT2Tokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = TFGPT2LMHeadModel.from_pretrained("datificate/gpt2-small-spanish")

Downloading: 100%|██████████| 498M/498M [02:08<00:00, 3.88MB/s]
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at datificate/gpt2-small-spanish.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
